<a href="https://colab.research.google.com/github/sw6820/STS/blob/master/STS_backtranslation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

In [ ]:
cd /content/drive/MyDrive/wanted/STS

In [ ]:
mkdir data

In [ ]:
cd ./data

In [ ]:
# !wget https://aistages-prod-server-public.s3.amazonaws.com/app/Competitions/000067/data/klue-sts-v1.1.tar.gz

In [ ]:
!pip install transformers
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!pip install pororo
!pip install folium==0.2.1 urllib3~=1.26

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
import numpy as np
import pandas as pd
import tarfile

import sys
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm
import os
import re
import time
from collections import Counter

import transformers
from transformers import BertTokenizer,AdamWeightDecay,TFRobertaModel,TFBertModel

import sklearn
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.model_selection import StratifiedKFold

import pororo
from pororo import Pororo

import torch 

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pickle
from tqdm.notebook import tqdm
from pororo import Pororo
# from datasets import load_from_disk
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# print(tf.__version__)
print(np.__version__)
print(pd.__version__)
print(transformers.__version__)
# print(keras.__version__)
print(sklearn.__version__)
print(pororo.__version__)
print(selenium.__version__)

In [ ]:
tar = tarfile.open("klue-sts-v1.1.tar.gz", "r:gz")
df = pd.read_json(tar.extractfile(tar.getmembers()[-1]))
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# df['annotations'][0]

In [ ]:
# df['labels'][0]

In [ ]:
df.duplicated(['sentence1', 'sentence2']).sum()

### Pororo BackTranslation

[github](https://github.com/kakaobrain/pororo)

[docs](https://kakaobrain.github.io/pororo/)

In [ ]:
mt = Pororo(task="translation", lang="multi")

we can choose 2 types of translation by [BLEU score](https://kakaobrain.github.io/pororo/seq2seq/mt.html)

ko, ja are above average BLUE scores


In [ ]:
def back_translation(s, lang) :
    transed_lang = mt(s, src = 'ko', tgt = lang) # ko to other lang
    ko = mt(transed_lang, src = lang, tgt = 'ko') # another to ko
    return ko

In [ ]:
def translated(lang):
  sentence1, sentence2 = [],[]  
  for idx, row in tqdm(df.iterrows()):
    df['sentence1'].append(back_translation(row['sentence1'], lang))
    df['sentence2'].append(back_translation(row['sentence2'], lang))
  return sentence1, sentence2

In [ ]:
en_sentence1_trans, en_sentence2_trans = translated('en')
ja_sentence1_trans, ja_sentence2_trans = translated('ja')

In [ ]:
en_sentence1 = pd.DataFrame(en_sentence1_trans)
en_sentence2 = pd.DataFrame(en_sentence2_trans)

ja_sentence1 = pd.DataFrame(ja_sentence1_trans)
ja_sentence2 = pd.DataFrame(ja_sentence2_trans)

en_sentence1_trans.to_csv('en_sentence1.csv')
en_sentence2_trans.to_csv('en_sentence2.csv')

ja_sentence1_trans.to_csv('ja_sentence1.csv')
ja_sentence2_trans.to_csv('ja_sentence2.csv')

### 위에 방식이 터지면 아래와 같이 잘라서 시도해보세요

In [ ]:
# half = len(df)//2
# for i in range(2):
#   sentence1, sentence2 = [],[]  

#   for idx, row in tqdm(df[half*i : half*(i+1)].iterrows()):
#     sentence1.append(back_translation(row['sentence1'], 'en'))
#     sentence2.append(back_translation(row['sentence2'], 'en'))

#   sentence1 = pd.DataFrame(sentence1)
#   sentence2 = pd.DataFrame(sentence2) 

#   en_sentence1_trans.to_csv(f'en_sentence1_{i+1}.csv')
#   en_sentence2_trans.to_csv(f'en_sentence2_{i+1}.csv')
